In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import sys; sys.path.insert(0, '../')
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import pearsonr
import mne
from esinet import Simulation
from esinet.forward import get_info, create_forward_model
from esinet.util import unpack_fwd
pp = dict(surface='white', hemi='both')

# Get Forward Model

In [2]:
info = get_info(kind='biosemi64')
fwd = create_forward_model(info=info, sampling='ico3')

leadfield, pos = unpack_fwd(fwd)[1:3]
n_chans, n_dipoles = leadfield.shape

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    2.1s remaining:    2.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.3s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s finished


# Get sample data

In [3]:
settings = dict(number_of_sources=3, extents=(25, 40), duration_of_trial=0.01, target_snr=25)

sim = Simulation(fwd, info, settings).simulate(2)
stc = sim.source_data[0]
evoked = sim.eeg_data[0].average()

brain = stc.plot(**pp)
brain.add_text(0.1, 0.9, 'Ground Truth', 'title',
               font_size=14)

-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


  0%|          | 0/2 [00:00<?, ?it/s]c:\Users\Lukas\Envs\invertenv\lib\site-packages\esinet\simulation.py:387: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.simulation_info = self.simulation_info.append(d, ignore_index=True)
 50%|█████     | 1/2 [00:00<00:00,  2.15it/s]c:\Users\Lukas\Envs\invertenv\lib\site-packages\esinet\simulation.py:387: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.simulation_info = self.simulation_info.append(d, ignore_index=True)
100%|██████████| 2/2 [00:00<00:00, 222.78it/s]


source data shape:  (1284, 1000) (1284, 1000)


100%|██████████| 2/2 [00:00<00:00, 10.34it/s]


Using pyvistaqt 3d backend.

Using control points [1.85667827e-08 2.85081228e-08 7.74159832e-08]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`


# Regularization Optimizations

In [ ]:
from invert.evaluate import corr, nmse
from invert.solvers.lucas import SolverLUCAS

solver = SolverLUCAS()
solver.make_inverse_operator(fwd, evoked, verbose=0)
solver.optimize_weights(fwd, info)
solver.plot_weights()
stc_hat = solver.apply_inverse_operator(evoked)
stc_hat.plot(**pp, brain_kwargs=dict(title=solver.name))

print("Mean correlation: ", np.mean(corr(stc_hat.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat.data, stc.data)))

# Temporal Context
# stc_hat_cbd = contextualize_bd(stc_hat, fwd, fast=True, lstm_look_back=10, verbose=1)
# stc_hat_cbd.plot(**pp, brain_kwargs=dict(title="c"+solver.name))

# print("Mean correlation: ", np.mean(corr(stc_hat_cbd.data, stc.data)))
# print("Mean NMSE: ", np.mean(nmse(stc_hat_cbd.data, stc.data)))


# # FOCUSS:
# stc_hat_focuss = focuss(stc_hat_cbd, evoked, fwd)
# stc_hat_focuss.plot(**pp, brain_kwargs=dict(title="c"+solver.name+" FOCUSS"))

# print("Mean correlation: ", np.mean(corr(stc_hat_focuss.data, stc.data)))
# print("Mean NMSE: ", np.mean(nmse(stc_hat_focuss.data, stc.data)))

In [5]:
from invert import Solver
from invert.evaluate import corr, nmse
from invert.adapters import contextualize_bd, focuss, s_focuss

solver_ = Solver(solver="laur")

solver_.make_inverse_operator(fwd, alpha="auto")
stc_hat = solver_.apply_inverse_operator(evoked)
# stc_hat.data[np.isnan(stc_hat.data)] = 0
stc_hat.plot(**pp, brain_kwargs=dict(title=solver_.name))
print("Mean correlation: ", np.mean(corr(stc_hat.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat.data, stc.data)))

stc_hat_sf = s_focuss(stc_hat, evoked, fwd, verbose=1)
stc_hat_sf.plot(**pp, brain_kwargs=dict(title=solver_.name + " shrinking FOCUSS"))
print("Mean correlation: ", np.mean(corr(stc_hat_sf.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat_sf.data, stc.data)))

stc_hat_sf = focuss(stc_hat, evoked, fwd, verbose=1)
stc_hat_sf.plot(**pp, brain_kwargs=dict(title=solver_.name + " FOCUSS"))
print("Mean correlation: ", np.mean(corr(stc_hat_sf.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat_sf.data, stc.data)))

# # Temporal Context
# stc_hat_cbd = contextualize_bd(stc_hat, fwd, fast=True, lstm_look_back=10, verbose=1)
# stc_hat_cbd.plot(**pp, brain_kwargs=dict(title="c"+solver_.name))

# print("Mean correlation: ", np.mean(corr(stc_hat_cbd.data, stc.data)))
# print("Mean NMSE: ", np.mean(nmse(stc_hat_cbd.data, stc.data)))


# # FOCUSS:
# stc_hat_focuss = focuss(stc_hat_cbd, evoked, fwd)
# stc_hat_focuss.plot(**pp, brain_kwargs=dict(title="c"+solver_.name+" FOCUSS"))

# print("Mean correlation: ", np.mean(corr(stc_hat_focuss.data, stc.data)))
# print("Mean NMSE: ", np.mean(nmse(stc_hat_focuss.data, stc.data)))


Using control points [1.00295540e-09 1.16641942e-09 2.91248426e-09]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Mean correlation:  0.14074433207742693
Mean NMSE:  0.023023068486939736
-- number of adjacent vertices : 1284
Shrinking FOCUSS:

Time step 0/10
Time step 1/10
Time step 2/10
Time step 3/10
Time step 4/10
Time step 5/10
Time step 6/10
Time step 7/10
Time step 8/10
Time step 9/10
Using control points [5.09529476e-59 1.25247081e-58 1.12653343e-55]
For automatic theme detection, "darkdetect" has to be installed! You can install it with `pip install darkdetect`
To use light mode, "qdarkstyle" has to be installed! You can install it with `pip install qdarkstyle`
Mean correlation:  -0.003055457928142368
Mean NMSE:  0.005667790421865522
FOCUSS:

Time step 0/10
Iteration 0
Iteration 1
Iteration 2
converged at repeti

# Adapt

In [4]:
from invert.adapters import contextualize_bd, contextualize

stc_hat_cbd = contextualize_bd(stc_hat, fwd, lstm_look_back=10, num_units=128, num_epochs=50, verbose=0)
stc_hat_cbd.plot(**pp, brain_kwargs=dict(title="c"+solver.name))

print("Mean correlation: ", np.mean(corr(stc_hat_cbd.data, stc.data)))
print("Mean NMSE: ", np.mean(nmse(stc_hat_cbd.data, stc.data)))

NameError: name 'stc_hat' is not defined

# Test/ Evaluation

In [5]:
from invert import Solver
from invert.config import all_solvers
from invert.evaluate import nmse, corr
from invert.adapters import contextualize_bd

settings = dict(number_of_sources=(1,10), extents=(1, 40), duration_of_trial=1, target_snr=(1,25))
errors = {sname: [] for sname in all_solvers}
solvers = dict()

for i in range(5):
    # print(i)
    sim = Simulation(fwd, info, settings).simulate(2)
    stc = sim.source_data[0]
    evoked = sim.eeg_data[0].average()

    for solver_name in all_solvers:
        print(solver_name)
        solver = Solver(solver=solver_name)
        if (not solver_name in solvers) or ("sparse" in solver_name.lower() or "bayes" in solver_name.lower()):
            solvers[solver_name] = solver.make_inverse_operator(fwd, evoked, alpha="auto")
        stc_hat = solvers[solver_name].apply_inverse_operator(evoked)
        # stc_hat.plot(**pp, brain_kwargs=dict(title=solver.name))
        error = np.mean(corr(stc.data, stc_hat.data))
        errors[solver_name].append( error )
        
        solver_name = "c" + solver_name
        if not solver_name in errors:
            errors[solver_name] = []
        stc_hat = contextualize_bd(stc_hat, fwd, fast=True)
        error = np.mean(corr(stc.data, stc_hat.data))
        
        errors[solver_name].append( error )




# import pickle as pkl
# fn = "errors.pkl"
# with open(fn, 'wb') as f:
#     pkl.dump(errors, f)

-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 182.31it/s]


source data shape:  (1284, 1000) (1284, 1000)


100%|██████████| 2/2 [00:00<00:00,  8.59it/s]


MNE
wMNE
dSPM
alpha must be set to a float when using Dynamic Statistical Parametric Mapping, auto does not work yet.
LORETA
sLORETA
eLORETA
LAURA
Backus-Gilbert
S-MAP
Multiple Sparse Priors
Bayesian LORETA
Bayesian MNE
Bayesian Beamformer
Bayesian Beamformer LORETA
Fully-Connected
-- number of adjacent vertices : 1284
Simulating data based on sparse patches.


...

100%|██████████| 2/2 [00:00<00:00, 334.23it/s]


source data shape:  (1284, 1000) (1284, 1000)


100%|██████████| 2/2 [00:00<00:00, 10.18it/s]


MNE
wMNE
dSPM


KeyboardInterrupt: 

In [6]:
errors

{'MNE': [0.34729599151247553, 0.27617302638060154],
 'wMNE': [0.09976123657625754, 0.05649753375297117],
 'dSPM': [0.14191310546497604, 0.10079960597715885],
 'LORETA': [0.39887611409150875],
 'sLORETA': [0.24520848218983327],
 'eLORETA': [0.28586065499599217],
 'LAURA': [0.258290045163993],
 'Backus-Gilbert': [0.14453254511835237],
 'S-MAP': [0.46738127561816595],
 'Multiple Sparse Priors': [0.17898519475252495],
 'Bayesian LORETA': [0.19208657907295815],
 'Bayesian MNE': [0.24019212025384604],
 'Bayesian Beamformer': [-0.001422764561104135],
 'Bayesian Beamformer LORETA': [0.45664365621374386],
 'Fully-Connected': [0.500778771518088],
 'LUCAS': [0.1449857433471954],
 'cMNE': [0.3911010166103456, 0.4422037079435315],
 'cwMNE': [0.09199144850186321, 0.06104926010298183],
 'cdSPM': [0.08250310070420695],
 'cLORETA': [0.5327607276422459],
 'csLORETA': [0.2814471635937683],
 'ceLORETA': [0.3525553572259107],
 'cLAURA': [0.20743491441016107],
 'cBackus-Gilbert': [0.23382860010129122],
 'cS

In [ ]:
import pandas as pd
import seaborn as sns
sns.set(font_scale=0.8)
df = pd.DataFrame(errors)
sorted_index = df.median().sort_values().index
df = df[sorted_index]

plt.figure()
sns.boxplot(data=df)
plt.title("Correlation with ground truth")


df_mean_var = pd.concat([df.mean(), df.std()], axis=1)
df_mean_var = df_mean_var.rename(columns={0: "Median", 1: "Variance"})
df_mean_var["MedVar"] = df_mean_var["Median"] / df_mean_var["Variance"]
df_mean_var["Method"] = df_mean_var.index
display(df_mean_var)

plt.figure()
sns.scatterplot(x="Median", y="Variance", hue="Method", size="MedVar", data=df_mean_var)
plt.xlabel("Median")
plt.ylabel("Variance")


In [ ]:
from invert.solvers.multiple_sparse_priors import SolverMultipleSparsePriors
from invert.solvers.loreta import SolverLORETA, SolverSLORETA, SolverELORETA
from invert.solvers.wrop import SolverBackusGilbert, SolverLAURA
from invert.solvers.smap import SolverSMAP
from invert.solvers.minimum_norm_estimates import SolverDynamicStatisticalParametricMapping, SolverWeightedMinimumNorm, SolverMinimumNorm
solvers = [SolverMultipleSparsePriors, SolverLORETA, SolverSLORETA, SolverELORETA, SolverBackusGilbert, SolverLAURA, SolverSMAP, SolverDynamicStatisticalParametricMapping, SolverWeightedMinimumNorm, SolverMinimumNorm]

for solver in solvers:
    solver_ = solver()
    if solver_.name == "Multiple Sparse Priors":
        solver_.make_inverse_operator(fwd, evoked, alpha='auto')
    else:
        solver_.make_inverse_operator(fwd, alpha='auto')
    stc_hat = solver_.apply_inverse_operator(evoked)
    stc_hat.plot(**pp, brain_kwargs=dict(title=solver_.name))



In [ ]:
from mne.minimum_norm import make_inverse_operator as mne_inverse
from mne.minimum_norm import apply_inverse as mne_apply
from mne import make_ad_hoc_cov
noise_cov = make_ad_hoc_cov(evoked.info, verbose=0)
mne_io = mne_inverse(evoked.info, fwd, noise_cov=noise_cov, fixed=True, loose=0, depth=0, verbose=0)
stc_hat = mne_apply(evoked, mne_io, method="MNE", verbose=0)
stc_hat.plot(**pp)